# Tutorial: Policy optimization algorithms

## Part 1: RL as black-box optimization

In this first part of the tutorial let's implement CEM and NES for a simple test function (defined below)

In [1]:
import numpy as np

# the function we want to optimize for sanity check
def f(theta):
  # Here would go the evaluation of the episode
  reward = -np.sum(np.square(solution - theta))
  return reward

solution = np.array([1, 1, -0.5])

# sanity check 
f(solution)

-0.0

In [4]:
#################################
# STARTER CODE - CEM
#################################
dim_theta = 3
theta_mean = np.zeros(dim_theta)
theta_std = np.ones(dim_theta)
batch_size = 25 # number of samples per batch
elite_frac = 0.2 # fraction of samples used as elite set


n_iter = 300
# Now, for the algorithms
for _ in range(n_iter):

    # Sample parameter vectors    

    #YOUR CODE HERE    

    # Get elite parameters
    
    #YOUR CODE HERE
    
    # Update theta_mean, theta_std
    
    #YOUR CODE HERE
    
    pass # remove/comment this line afterwards

In [5]:
#################################
# STARTER CODE - NES
#################################
w0 = np.random.randn(3) #initial guess
npop=50
n_iter=300
sigma=0.1
alpha=0.001

for _ in range(n_iter):
    # Sample vectors from a normal distribution
    
    # YOUR CODE HERE
    
    # Sample function values by evaluating on the population
    
    # YOUR CODE HERE
    
    # Optional: standardize (substract mean and divide by std)
    
    # "Gradient" update
    
    # YOUR CODE HERE
    
    pass # remove/comment this line afterwards
    

## Your turn

- How can you re-use your code above for `CartPole`/`FrozenLake`? 
- Try to solve one of these environments!

## Part 2: Montecarlo Policy Gradient

This is an example implementation of REINFORCE, as explained in the lecture.

In [1]:
import gym
import numpy as np

In [2]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
def featurize(s,a):
    return (2*a-1)*np.tanh(s)

def softmax_policy(theta, actions):
    def policy_fn(state):
        exp = np.exp([np.dot(featurize(state,a),theta) for a in actions])
        probs = exp/np.sum(exp)
        return probs
    return policy_fn

def run_episode(env, random_policy): 
    done = False
    total_reward = 0
    state = env.reset()
    episode = []
    actions = range(env.action_space.n)
    while not done:
        probs = random_policy(state)
        action = np.random.choice(actions, p=probs)
        new_state, reward, done, _ = env.step(action)
        episode.append((state,action,reward))
        state = new_state
        total_reward += reward
    return episode, total_reward


In [9]:
theta = 2*np.random.rand(4)-1
n_episodes = 10000
actions = range(env.action_space.n)
gamma = 1.
alpha = 1e-3

In [10]:
for ep in range(n_episodes):
    G = 0
    policy = softmax_policy(theta,actions)
    episode, ep_reward = run_episode(env,policy)
    
    if ep_reward == 200:
        print("Won! this happened in episode ", ep)
        break
    
    if (ep+1) % 1000 == 0:
        print("Reward in episode {}: {}".format(ep+1,ep_reward))
    
    
    for i, _ in enumerate(episode):
        s, a, r = _
        G = sum(x[2]*(gamma**j) for j, x in enumerate(episode[i:]))
        s = np.array(s)
        theta = theta + alpha*(featurize(s,a)-np.sum([policy(s)[a]*featurize(s,a) for a in actions]))*G


Reward in episode 1000: 99.0
Won! this happened in episode  1244


## Your turn:

- Implement REINFORCE and REINFORCE with baseline for `CliffWalking`.

## Additional material

If you feel ready for a more interesting challenge, you can try `Pong`, which is one of the simples Atari environments. You should do some preprocessing of the video frames, to speed up the computation (resize, downsample, and remove background or color altogether).





Try to solve this yourself first! If you get stuck or want some inspiration, look at Andrej Karpathy's post:

- http://karpathy.github.io/2016/05/31/rl/
- https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5